#***RAG using openAI***

In [ ]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/664.8 MB 116.2 MB/s eta 0:00:05
ERROR: Operation cancelled by user


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
sentences=[
    "I love cats",
    "Dogs are the best",
    "Artificial intelligence is the future",
    "cats are cute animals",
    "I enjoy machine machine learning",
    "i have a pet dog"
]

In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings=model.encode(sentences)  #shape(6,384)
embeddings=np.array(embeddings).astype('float32')

In [ ]:
dimension=embeddings.shape[1]
index=faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
query="I adore dogs"
query_vector=model.encode([query]).astype("float32")

top_k=3
distances,indices=index.search(query_vector,top_k)

In [ ]:
print("Query:", query)
print("\nTop Similar Sentences:")
for idx in indices[0]:
  print("-",sentences[idx])

Query: I adore dogs

Top Similar Sentences:
- Dogs are the best
- I love cats
- i have a pet dog


In [ ]:
import openai
!pip insrtall faiss-cpu sentence-transformers openai
openai.api_key="YOUR OPEN AI API KEY"

ERROR: unknown command "insrtall" - maybe you meant "install"


In [ ]:
retrieved_chunks = [sentences[idx] for idx in indices[0]]

In [ ]:
context="\n".join(retrieved_chunks)
prompt=f"""Answer the question based on the context below:
context:
{context}
question:
{query}
answer:
"""

client = openai.OpenAI(api_key=openai.api_key)

response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":"You are a helpful assistant."},
        {"role":"user","content":prompt}
    ]
)

print("\n  Final Answer:\n",response.choices[0].message.content.strip())


  Final Answer:
 Yes, based on the context provided, the statement "I adore dogs" would likely be true because you have expressed a positive sentiment towards dogs.
